In [3]:
import cv2
import json
import os
import numpy as np
import albumentations as A
from tqdm import tqdm

# Load your annotations
with open('hand_gesture_data/annotations.json', 'r') as f:
    annotations = json.load(f)

# Define augmentation pipeline
transform = A.Compose([
    A.RandomRotate90(p=0.5),
    A.Flip(p=0.5),
    A.Transpose(p=0.5),
    A.OneOf([
        A.GaussNoise(p=0.9),
        A.GaussNoise(),
    ], p=0.2),
    A.OneOf([
        A.MotionBlur(p=0.2),
        A.MedianBlur(blur_limit=3, p=0.1),
        A.Blur(blur_limit=3, p=0.1),
    ], p=0.2),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
    A.OneOf([
        A.OpticalDistortion(p=0.3),
        A.GridDistortion(p=0.1),
        A.PiecewiseAffine(p=0.3),
    ], p=0.2),
    A.OneOf([
        A.CLAHE(clip_limit=2),
        A.Sharpen(),
        A.Emboss(),
        A.RandomBrightnessContrast(),
    ], p=0.3),
    A.HueSaturationValue(p=0.3),
], bbox_params=A.BboxParams(format='coco', label_fields=['class_labels']))

# Function to augment a single image
def augment_image(image, bbox, class_label, num_augmentations=9):
    augmented_images = []
    augmented_bboxes = []
    
    for _ in range(num_augmentations):
        augmented = transform(image=image, bboxes=[bbox], class_labels=[class_label])
        augmented_images.append(augmented['image'])
        augmented_bboxes.append(augmented['bboxes'][0])
    
    return augmented_images, augmented_bboxes

# Augment the dataset
new_annotations = {}

for img_name, annotation in tqdm(annotations.items()):
    # Load the original image
    img_path = os.path.join('hand_gesture_data', img_name)
    print(img_path)
    # if (img_path == "hand_gesture_data/closed_fist_0.jpg" or img_path == "hand_gesture_data/closed_fist_1.jpg"):
    #     continue;
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Get the original bbox and class
    bbox = annotation['bbox']
    print(bbox)
    class_label = annotation['gesture']
    
    # Add the original image to new annotations
    new_annotations[img_name] = annotation
    
    # Augment the image
    aug_images, aug_bboxes = augment_image(image, bbox, class_label)
    
    # Save augmented images and add to new annotations
    for i, (aug_image, aug_bbox) in enumerate(zip(aug_images, aug_bboxes)):
        aug_img_name = f"{os.path.splitext(img_name)[0]}_aug_{i}.jpg"
        aug_img_path = os.path.join('hand_gesture_data', aug_img_name)
        
        # Save the augmented image
        cv2.imwrite(aug_img_path, cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR))
        
        # Add to new annotations
        new_annotations[aug_img_name] = {
            'gesture': class_label,
            'bbox': list(map(int, aug_bbox))  # Convert float to int
        }

# Save the new annotations
with open('hand_gesture_data/augmented_annotations.json', 'w') as f:
    json.dump(new_annotations, f)

print(f"Augmentation complete. Total images: {len(new_annotations)}")

  1%|          | 1/126 [00:00<00:22,  5.46it/s]

hand_gesture_data/thumbs_up_0.jpg
[252, 339, 662, 699]
hand_gesture_data/thumbs_up_1.jpg
[423, 277, 408, 657]


  2%|▏         | 2/126 [00:02<02:56,  1.42s/it]

hand_gesture_data/thumbs_up_2.jpg
[337, 191, 708, 875]


  2%|▏         | 3/126 [00:02<01:54,  1.07it/s]

...

hand_gesture_data/pointing_finger_25.jpg
[344, 529, 690, 527]


 99%|█████████▉| 125/126 [02:39<00:01,  1.05s/it]

hand_gesture_data/pointing_finger_26.jpg
[266, 307, 991, 771]


100%|██████████| 126/126 [02:44<00:00,  1.30s/it]

Augmentation complete. Total images: 1260
